In [ ]:
import os

In [ ]:
# file_name = "extraitalice"
# file_name = "alice"
file_name = "textesimple"
with open(file_name +"/"+file_name + ".txt") as f:
    text = f.read()

In [ ]:
list_sorted_label = []

for i in text:
    list_sorted_label.append(i)

list_sorted_label.sort()

print(list_sorted_label)

In [ ]:
list_label = []
list_freq = []

for i in list_sorted_label:
    if i not in list_label:
        list_label.append(i)
        list_freq.append(list_sorted_label.count(i))

print(list_label)
print(list_freq)

In [ ]:
list_sorted_freq = zip(list_freq,list_label)
list_sorted_freq = sorted(list_sorted_freq)
list_sorted_freq = zip(*list_sorted_freq)

list_freq, list_label = [[*tuple] for tuple in list_sorted_freq]

print(list_freq)
print(list_label)

In [ ]:
length_dict = len(list_label)
text_freq = str(length_dict) + "\n\n"

for i in range (length_dict):
    text_freq = text_freq + list_label[i] + " " + str(list_freq[i]) + "\n"

print(text_freq)

In [ ]:
if os.path.exists(file_name +"/"+file_name + '_freq.txt'):
    os.remove(file_name +"/"+file_name + '_freq.txt')

new_file = open(file_name +"/"+file_name + "_freq.txt", "w")
new_file.write(text_freq)
new_file.close()

In [ ]:
class node:
    def __init__(self, label, freq, left_child=None, right_child=None):
         # Label de la lettre : "a"
        self.label = label
 
        # Fréquence de la lettre : 10
        self.freq = freq
 
        # L'enfant gauche(0) de la lettre en question : de type node
        self.left_child = left_child
 
        # L'enfant droit(1) de la lettre en question : de type node
        self.right_child = right_child
 
        # Direction : 0 ou 1
        self.direction = ''

    def __str__(self):
        return str(self.label) + " " + str(self.freq) + " " + str(self.left_child) + " " + str(self.right_child)

In [ ]:
list_node = []

for i in range(len(list_label)):
    list_node.append(node(list_label[i], list_freq[i]))

for i in range(len(list_node)):
    print(list_node[i])

In [ ]:
while len(list_node) > 1:

    list_node = sorted(list_node, key=lambda x: x.freq)
 
    # Affectation des deux noeuds les plus petits
    node_left = list_node[0]
    node_right = list_node[1]
 
    # Affectation d'une direction à ces deux noeuds
    node_left.direction = 0
    node_right.direction = 1

    freq_parent = node_left.freq + node_right.freq

    node_parent = node(freq_parent,freq_parent , node_left, node_right)
 
 
    # Suppression des noeud enfant pour ne garder que le noeud parent
    list_node.remove(node_left)
    list_node.remove(node_right)
    list_node.append(node_parent)

print(list_node[0])

In [ ]:
def createListNodes(node, binary, list):
    binary = binary + str(node.direction) 
    dict_comp = {}

    if (node.left_child):
        createListNodes(node.left_child, binary,list)

    if (node.right_child):
        createListNodes(node.right_child, binary,list)

    if(not node.left_child and not node.right_child):
        dict_comp[node.label] = binary
        list.append(dict_comp)
    return list 

In [ ]:
binary = ""
list = []

list_nodes = createListNodes(list_node[0], binary,list )

print(list_nodes)

In [ ]:
for key in list_nodes[0].keys():
    print (key)

for value in list_nodes[0].values():
    print (value)

for key,value in list_nodes[0].items():
    print(key, value)

In [ ]:
list_text = []

for i in text:
    list_text.append(i)
    
print(list_text)

In [ ]:
for i in range(len(list_text)):
    for j in list_nodes:
        for cle,valeur in j.items():
            if (cle == list_text[i]):
                list_text[i] = valeur
print(list_text)

In [ ]:
text_binary = ""
for i in list_text:
    text_binary = text_binary + i

print(text_binary)

In [ ]:
text_compressed = int(text_binary, base=2).to_bytes((len(text_binary)+7)//8, byteorder='big') 
print(text_compressed)